### GALAXY PAIRS CATALOGS

In [77]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from astropy.io import fits
from astropy.cosmology import Planck18 as cosmo
from astropy.coordinates import SkyCoord
import astropy.units as u
import healpy as hp
from matplotlib import rc_params_from_file
from tqdm.notebook import tqdm
from scipy.ndimage import gaussian_filter

In [78]:
import numpy as np

def fast_icrs_to_galactic(ra_deg, dec_deg):
    """
    Convert ICRS coordinates (RA, Dec in degrees) to Galactic (l, b in degrees)
    without using astropy. Based on standard transformation matrix.

    Parameters:
        ra_deg : float or array-like
            Right Ascension in degrees
        dec_deg : float or array-like
            Declination in degrees

    Returns:
        l_deg, b_deg : tuple of arrays
            Galactic longitude and latitude in degrees
    """
    # Convert to radians
    ra = np.radians(ra_deg)
    dec = np.radians(dec_deg)

    # Convert spherical to Cartesian
    x = np.cos(dec) * np.cos(ra)
    y = np.cos(dec) * np.sin(ra)
    z = np.sin(dec)

    # Rotation matrix from ICRS to Galactic
    R = np.array([
        [-0.0548755604, -0.8734370902, -0.4838350155],
        [ 0.4941094279, -0.4448296300,  0.7469822445],
        [-0.8676661490, -0.1980763734,  0.4559837762]
    ])

    # Apply rotation
    xg, yg, zg = np.dot(R, np.array([x, y, z]))

    # Convert back to spherical
    b_rad = np.arcsin(zg)
    l_rad = np.arctan2(yg, xg)

    # Convert to degrees
    l_deg = np.degrees(l_rad) % 360
    b_deg = np.degrees(b_rad)

    return l_deg, b_deg

In [79]:
# --- Settings ---
catalog, region = "LRG", "NGC"
real_file = f"data/eBOSS/eBOSS_{catalog}_clustering_data-{region}-vDR16.fits"
rand_file = f"data/eBOSS/eBOSS_LRG_clustering_random-{region}-vDR16.fits"
alm_file = "data/COM_Lensing_4096_R3.00/MV/dat_klm.fits"
mask_file = "data/COM_Lensing_4096_R3.00/mask.fits"

In [80]:
# --- Catalog loader ---
def load_catalog(path, weights=True, random_fraction=None):
    with fits.open(path) as hd:
        cat = hd[1].data
    cat = cat[(cat['Z'] > 0) & np.isfinite(cat['RA']) & np.isfinite(cat['DEC'])]
    if random_fraction:
        cat = cat[np.random.choice(len(cat), int(random_fraction * len(cat)), replace=False)]
    w = np.ones(len(cat)) if not weights else cat['WEIGHT_NOZ'] * cat['WEIGHT_SYSTOT']
    return cat, w

In [81]:
def preprocess_catalog_galactic(data):
    z = data['Z']
    ra = data['RA']
    dec = data['DEC']
    D = cosmo.comoving_distance(z).value
    valid = (D > 0) & np.isfinite(D)

    ra_valid = ra[valid]
    dec_valid = dec[valid]
    D_valid = D[valid]

    l, b = fast_icrs_to_galactic(ra_valid, dec_valid)

    return l, b, D_valid, data[valid]

In [82]:
import pandas as pd

def compute_angle_cosine(l1, b1, l2, b2):
    """
    Compute cosine of angle θ between two directions (in degrees) in Galactic coordinates.
    """
    l1_rad, b1_rad = np.radians(l1), np.radians(b1)
    l2_rad, b2_rad = np.radians(l2), np.radians(b2)

    cos_theta = (
        np.cos(b1_rad) * np.cos(l1_rad) * np.cos(b2_rad) * np.cos(l2_rad) +
        np.cos(b1_rad) * np.sin(l1_rad) * np.cos(b2_rad) * np.sin(l2_rad) +
        np.sin(b1_rad) * np.sin(b2_rad)
    )
    return cos_theta

def build_galaxy_pair_catalog(data, weights, r_par_max=5, r_perp_min=4, r_perp_max=6):
    """
    Build galaxy pair catalog based on parallel and perpendicular distance criteria.
    Returns a list of dictionaries.
    """
    l, b, D, data_filtered = preprocess_catalog_galactic(data)
    weights = weights[:len(data_filtered)]
    z = data_filtered['Z']

    pairs = []

    for i in tqdm(range(len(data_filtered)), desc="Building Pair Catalog"):
        for j in range(i + 1, len(data_filtered)):
            Dc1, Dc2 = D[i], D[j]
            r_par = np.abs(Dc2 - Dc1)  # cos(θ) ≈ 1

            if r_par > r_par_max:
                continue

            cos_theta = compute_angle_cosine(l[i], b[i], l[j], b[j])
            theta = np.arccos(np.clip(cos_theta, -1, 1))

            r_perp = (Dc2 + Dc1) * theta  # sin(θ) ≈ θ

            if r_perp_min <= r_perp <= r_perp_max:
                Dmid = cosmo.comoving_distance((z[i] + z[j]) / 2).value
                pairs.append({
                    'l1': l[i], 'b1': b[i], 'z1': z[i], 'w1': weights[i], 'Dc1': Dc1, 'LRG_ID1': data_filtered[i]['LRG_ID'],
                    'l2': l[j], 'b2': b[j], 'z2': z[j], 'w2': weights[j], 'Dc2': Dc2, 'LRG_ID2': data_filtered[j]['LRG_ID'],
                    'Dmid': Dmid
                })

    pairs = pd.DataFrame(pairs)
    pairs.to_csv("data/galaxy_pairs_catalog.csv", index=False)
    return pairs

In [83]:
# --- Run all ---
data_real, w_real = load_catalog(real_file, weights=True)
data_rand, w_rand = load_catalog(rand_file, random_fraction=0.10)

# Build galaxy pair catalog
pair_catalog = build_galaxy_pair_catalog(data_real, w_real)
print(f"Total valid pairs: {len(pair_catalog)}")

# Jackknife real galaxies
# kappa_real, sigma_real = jackknife_stack_healpix(data_real, w_real, data_rand, nside=10)
# sn_real = np.zeros_like(kappa_real)
# valid = sigma_real > 0
# sn_real[valid] = kappa_real[valid] / sigma_real[valid]
# kappa_rand, sigma_rand, sn_rand = stack_kappa(data_rand, w_rand, "Random")
# kappa_sub = kappa_real - kappa_rand
# kappa_smooth = gaussian_filter(kappa_sub, sigma=2)

Building Pair Catalog:   0%|          | 0/107500 [00:00<?, ?it/s]

Total valid pairs: 1272
